****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

###  Import the libraries and parse in the xml document as an Element tree

In [1]:
import numpy as np
import pandas as pd
from xml.etree import ElementTree as ET

In [2]:
document = ET.parse( './data/mondial_database.xml' )

*****
### Exercise 1: Find 10 countries with the lowest mortality rate

In [3]:
# create a dictionary for infant mortality data by country 
data = dict()

# iterate over all countries and get infant mortality , if not available record NaN
for element in document.iterfind('country'):
    country_name = element.find('name').text 
    subelement = element.find('infant_mortality')
    if subelement is None:
        infant_mortality = np.NaN
    else:
        infant_mortality = float(subelement.text)
    data[country_name] = infant_mortality

In [4]:
# create a dataframe from the data dictionary
df = pd.DataFrame.from_dict(data, orient='index')
df.columns = ['infant_mortality']
df.head()

,infant_mortality
Canada,4.71
Brazil,19.21
Sao Tome and Principe,49.16
Turkmenistan,38.13
Saint Helena,17.63


In [5]:
# output the countries with lowest mortality 
print '10 Countries with Lowest Infant Mortality rate'
print df.sort_values('infant_mortality').head(10)

10 Countries with Lowest Infant Mortality rate
                infant_mortality
Monaco                      1.81
Japan                       2.13
Norway                      2.48
Bermuda                     2.48
Singapore                   2.53
Sweden                      2.60
Czech Republic              2.63
Hong Kong                   2.73
Macao                       3.13
Iceland                     3.15


*****
### Exercise 2: Find 10 cities with largest population

In [6]:
# create a dictionary of population by (city, year ) key 
data = dict() 

# iterate over all cities in the XML tree
for element in document.getiterator('city'):
    city = element.find("name").text
    for subelement in element.iterfind('population'):
        year = int(subelement.get("year"))
        population = float(subelement.text)
        data[(city,year)] = population

In [7]:
# create a dataframe with city populations by year
df = pd.DataFrame.from_dict(data, orient = 'index')
df.index = pd.MultiIndex.from_tuples(df.index)
df = df.reset_index()
df.columns = ['city', 'year', 'population']
df.head()

,city,year,population
0,Saarbrücken,2011,175853.0
1,Patra,1991,190463.0
2,Asyut,1996,343662.0
3,Comayagua,2001,87474.0
4,Cheltenham,2011,116447.0


In [8]:
# sort dataframe by city, and years in descending order
df = df.sort_values(['city', 'year'])
df.head(10)

,city,year,population
9077,'s-Hertogenbosch,1995,199127.0
5483,'s-Hertogenbosch,2004,133511.0
4822,'s-Hertogenbosch,2010,139754.0
3443,'s-Hertogenbosch,2014,143822.0
1958,A Coruña,1991,243134.0
6963,A Coruña,1994,255087.0
3878,A Coruña,2001,236379.0
1873,A Coruña,2011,245053.0
5718,Aachen,1987,229740.0
4456,Aachen,2011,236420.0


In [9]:
# group by city and take the last record, which will be the latest year 
df = df.groupby('city')['population'].last()
print "Top 10 Cities by Population"
df.sort_values(ascending = False).head(10)

Top 10 Cities by Population


city
Shanghai     22315474.0
Istanbul     13710512.0
Mumbai       12442373.0
Moskva       11979529.0
Beijing      11716620.0
São Paulo    11152344.0
Tianjin      11090314.0
Guangzhou    11071424.0
Delhi        11034555.0
Shenzhen     10358381.0
Name: population, dtype: float64

*****
### Exercise 3: Find 10 ethnic groups with the largest overall populations (sum of latest estimates over all countries)

#### Read in population and ethnic group data from xml into python dictionaries

In [10]:
# create a dictionary of population by (city, year ) key 
population_dict = dict() 
ethnicgroup_dict = dict()

# iterate over all countries in the XML tree
for element in document.getiterator('country'):
    country = element.find("name").text
    # get population records 
    for subelement in element.iterfind('population'):
        year = int(subelement.get("year"))
        population = float(subelement.text)
        population_dict[(country,year)] = population
    # get ethnic groups and percentages
    for subelement in element.iterfind('ethnicgroup'):
        group = subelement.text
        percentage = float(subelement.get('percentage'))
        ethnicgroup_dict[(country,group)] = percentage
    

#### Create a dataframe with the latest estimates of country popupations

In [11]:
# create a dataframe with population in the country by year
c = pd.DataFrame.from_dict(population_dict, orient = 'index')
c.index = pd.MultiIndex.from_tuples(c.index)
c = c.reset_index()
c.columns = ['country', 'year', 'population']
c = c.sort_values(['country', 'year'])
c.head()

,country,year,population
2042,Afghanistan,1950,7450738.0
1566,Afghanistan,1960,8774440.0
1953,Afghanistan,1970,11015621.0
1301,Afghanistan,1979,13051358.0
728,Afghanistan,2006,22575900.0


In [12]:
# get the latest year for when population was recorded
latest_year = c.groupby('country')['year'].max().to_frame()
latest_year = latest_year.reset_index()
latest_year.head()

,country,year
0,Afghanistan,2013
1,Albania,2011
2,Algeria,2010
3,American Samoa,2010
4,Andorra,2011


In [13]:
# merge with country populations and get the latest population
c = pd.merge(latest_year,c, on =['country','year'])
c = c.drop('year', axis = 1)
c.head()

,country,population
0,Afghanistan,26023100.0
1,Albania,2800138.0
2,Algeria,37062820.0
3,American Samoa,55519.0
4,Andorra,78115.0



#### Create a dataframe with percentage of ethnic groups in each country 

In [14]:
e = pd.DataFrame.from_dict(ethnicgroup_dict, orient = 'index')
e.index = pd.MultiIndex.from_tuples(e.index)
e = e.reset_index()
e.columns = ['country', 'group', 'percentage']
e.head()

,country,group,percentage
0,Myanmar,Karen,7.0
1,Serbia,Croat,1.1
2,Czech Republic,Slovak,1.9
3,Cote dIvoire,Bete,18.0
4,Ecuador,Amerindian,25.0


#### Merge data to get population by ethnic group

In [15]:
# combine ethnic group data with country population
df = pd.merge(e,c, on=['country'] )
df = df.rename(columns = {'population':'country_population'})
df.head()

,country,group,percentage,country_population
0,Myanmar,Karen,7.0,51419420.0
1,Myanmar,Mon,2.0,51419420.0
2,Myanmar,Chinese,3.0,51419420.0
3,Myanmar,Shan,9.0,51419420.0
4,Myanmar,Burman,68.0,51419420.0


In [16]:
# calculate population by ethnicity group
df['population'] = df['country_population']*df['percentage']/100
df.head()

,country,group,percentage,country_population,population
0,Myanmar,Karen,7.0,51419420.0,3599359.4
1,Myanmar,Mon,2.0,51419420.0,1028388.4
2,Myanmar,Chinese,3.0,51419420.0,1542582.6
3,Myanmar,Shan,9.0,51419420.0,4627747.8
4,Myanmar,Burman,68.0,51419420.0,34965205.6


In [17]:
# aggregate population for each group
res = df.groupby('group')['population'].sum()
print "Top 10 Ethnic Groups by Population"
res.sort_values(ascending= False).head(10)

Top 10 Ethnic Groups by Population


group
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: population, dtype: float64

*****
### Excercise 4 a : Find name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [18]:
# rivers
data_dict = dict()

# iterate through al rivers and record the names, countries and length in the dictionary
for element in document.iterfind('river'):
    name =  element.find('name').text
    subelement =  element.find('length')
    if subelement is None:
        length = np.NaN
    else:
        length = float(subelement.text)
    country =  element.get('country')
    data_dict[name]= [country, length]

In [19]:
#create a dataframe
rivers = pd.DataFrame.from_dict(data_dict,orient = 'index')
rivers.columns = ['country', 'length']
rivers.head()

,country,length
Karun,IR,950.0
Waag,SK,403.0
Mosel,D L F,544.0
Oesterdalaelv,S,241.0
Theiss,H UA SRB,1308.0


In [20]:
# get the longest river 
print 'Longest river'
rivers.sort_values('length', ascending = False).head(1)

Longest river


,country,length
Amazonas,CO BR PE,6448.0


*****
### Excercise 4b: Find name and country of  largest lake

In [21]:
# lakes
data_dict = dict()

# iterate through al rivers and record the names, countries and length in the dictionary
for element in document.iterfind('lake'):
    name =  element.find('name').text
    subelement =  element.find('area')
    if subelement is None:
        area = np.NaN
    else:
        area = float(subelement.text)
    country =  element.get('country')
    data_dict[name]= [country, area]

In [22]:
# create a dataframe with lakes countries and areas
lakes = pd.DataFrame.from_dict(data_dict,orient = 'index')
lakes.columns = ['country', 'area']
lakes.head()

,country,area
Vierwaldstattersee,CH,113.7
Ammersee,D,46.6
Lake Manicouagan,CDN,1942.0
Lake Erie,CDN USA,25745.0
Pyramid Lake,USA,487.0


In [23]:
# get the largest lake 
print 'Largest Lake'
lakes.sort_values('area', ascending = False).head(1)

Largest Lake


,country,area
Caspian Sea,R AZ KAZ IR TM,386400.0


*****
### Excercise 4c: Find name and country of  airport at highest elevation

In [24]:
# airports
data_dict = dict()

# iterate through al rivers and record the names, countries and length in the dictionary
for element in document.iterfind('airport'):
    name =  element.find('name').text
    elevation  =  element.find('elevation').text
    country =  element.get('country')
    data_dict[name]= [country, elevation]

In [25]:
# create a dataframe with airoports height
airports = pd.DataFrame.from_dict(data_dict,orient = 'index')
airports.columns = ['country', 'elevation']

# remove nulls from elevation column and convert it to numeric
airports = airports.dropna()
airports['elevation'] = pd.to_numeric(airports['elevation'])

airports.head()

,country,elevation
Pierre Elliott Trudeau Intl,CDN,36
Rovaniemi,SF,196
Jerez,E,28
Nampula,MOC,440
Yibin,CN,305


In [26]:
# get the highest airport 
print "Highest airport" 
airports.sort_values('elevation', ascending = False).head(1)

Highest airport


,country,elevation
El Alto Intl,BOL,4063
